In [1]:
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from codecarbon import EmissionsTracker
from ptflops import get_model_complexity_info
import os
import pandas as pd

In [2]:
# ----------------------------
# Set Seed
# ----------------------------
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
# ----------------------------
# ShuffleNet-V2 Deepened
# ----------------------------
class ShuffleNetV2_Deepened_Unfrozen(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = models.shufflenet_v2_x1_0(weights='IMAGENET1K_V1')

        # Unfreeze all base layers
        for param in self.base_model.parameters():
            param.requires_grad = True


        self.extra_layers = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # Replace classifier
        self.classifier = nn.Linear(self.base_model.fc.in_features, num_classes)

    def forward(self, x):
        x = self.base_model.conv1(x)
        x = self.base_model.maxpool(x)
        x = self.base_model.stage2(x)
        x = self.base_model.stage3(x)
        x = self.base_model.stage4(x)
        x = self.base_model.conv5(x)
        x = self.extra_layers(x)
        x = x.mean([2, 3])  # Global Average Pooling
        x = self.classifier(x)
        return x

In [4]:
# ----------------------------
# Dataset & DataLoader
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset   = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:04<00:00, 34.8MB/s]


In [5]:
# ----------------------------
# Training & Validation
# ----------------------------
def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

def validate(model, val_loader, criterion):
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return val_loss / total, accuracy

In [6]:
# ----------------------------
# Main Training Loop
# ----------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_classes = 10
results = []

output_dir = "./emissions_shufflenet_unfrozen"
os.makedirs(output_dir, exist_ok=True)

for seed in [0, 1, 2, 3, 4]:
    print(f"\n🌱 Running Seed {seed}")
    set_seed(seed)

    model = ShuffleNetV2_Deepened_Unfrozen(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # FLOPs Calculation
    macs, params = get_model_complexity_info(model, (3, 32, 32),
                                             as_strings=True,
                                             print_per_layer_stat=False,
                                             verbose=False)
    macs_value = macs.replace('Mac', '').strip()
    if 'G' in macs_value:
        macs_num = float(macs_value.replace('G', '')) * 2
        flops_per_forward = f"{macs_num:.2f} GFLOPs"
    elif 'M' in macs_value:
        macs_num = float(macs_value.replace('M', '')) / 1000 * 2
        flops_per_forward = f"{macs_num:.3f} GFLOPs"
    else:
        flops_per_forward = f"{macs_value} MACs"
    print(f"Seed {seed} → FLOPs per forward pass: {flops_per_forward} | Params: {params}")

    # Energy & CO₂ Tracking
    tracker = EmissionsTracker(
        project_name=f"ShuffleNet_Unfrozen_CIFAR10_Seed{seed}",
        output_dir=output_dir,
        measure_power_secs=1
    )
    tracker.start()
    start_time = time.time()

    # Training Loop (4 epochs)
    for epoch in range(4):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = validate(model, val_loader, criterion)
        print(f"Seed {seed} | Epoch {epoch+1}/4 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")

    emissions = tracker.stop()
    elapsed = time.time() - start_time

    results.append((seed, emissions, elapsed, flops_per_forward))


🌱 Running Seed 0
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth


100%|██████████| 8.79M/8.79M [00:00<00:00, 122MB/s]
[codecarbon WARNING @ 02:14:33] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 02:14:33] [setup] RAM Tracking...
[codecarbon INFO @ 02:14:33] [setup] CPU Tracking...


Seed 0 → FLOPs per forward pass: 0.044 GFLOPs | Params: 21.17 M


[codecarbon WARNING @ 02:14:34] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 02:14:34] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 02:14:34] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 02:14:34] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 02:14:34] [setup] GPU Tracking...
[codecarbon INFO @ 02:14:34] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 02:14:34] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 02:14:34] >>> Tracker's metadata:
[codecarbon INFO @ 02:14:34]   Platform sys

Seed 0 | Epoch 1/4 | Train Loss: 0.4801 | Val Acc: 91.30%


...

Seed 4 | Epoch 2/4 | Train Loss: 0.1901 | Val Acc: 92.62%


[codecarbon INFO @ 03:23:35] Energy consumed for RAM : 0.001261 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 03:23:35] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 03:23:35] Energy consumed for All CPU : 0.005371 kWh
[codecarbon INFO @ 03:23:35] Energy consumed for all GPUs : 0.007367 kWh. Total GPU Power : 47.1933694174917 W
[codecarbon INFO @ 03:23:35] 0.014000 kWh of electricity used since the beginning.
[codecarbon INFO @ 03:23:36] Energy consumed for RAM : 0.001264 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 03:23:36] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 03:23:36] Energy consumed for All CPU : 0.005383 kWh
[codecarbon INFO @ 03:23:36] Energy consumed for all GPUs : 0.007384 kWh. Total GPU Power : 58.92039232235113 W
[codecarbon INFO @ 03:23:36] 0.014031 kWh of electricity used since the beginning.
[codecarbon INFO @ 03:23:37] Energy consumed for RAM : 0.001267 kWh. RAM Power : 10

Seed 4 | Epoch 3/4 | Train Loss: 0.1166 | Val Acc: 93.26%


[codecarbon INFO @ 03:27:26] Energy consumed for RAM : 0.001891 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 03:27:26] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 03:27:26] Energy consumed for All CPU : 0.008051 kWh
[codecarbon INFO @ 03:27:26] Energy consumed for all GPUs : 0.011043 kWh. Total GPU Power : 52.165118767162916 W
[codecarbon INFO @ 03:27:26] 0.020985 kWh of electricity used since the beginning.
[codecarbon INFO @ 03:27:27] Energy consumed for RAM : 0.001894 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 03:27:27] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 03:27:27] Energy consumed for All CPU : 0.008063 kWh
[codecarbon INFO @ 03:27:27] Energy consumed for all GPUs : 0.011058 kWh. Total GPU Power : 54.207206487985275 W
[codecarbon INFO @ 03:27:27] 0.021014 kWh of electricity used since the beginning.
[codecarbon INFO @ 03:27:28] Energy consumed for RAM : 0.001896 kWh. RAM Power :

Seed 4 | Epoch 4/4 | Train Loss: 0.0747 | Val Acc: 93.42%


In [8]:
# ----------------------------
# Post-processing Results
# ----------------------------
csv_file = os.path.join(output_dir, "emissions.csv")
df = pd.read_csv(csv_file)

results_with_energy = []

for seed, emissions, elapsed, flops in results:
    project_name = f"ShuffleNet_Unfrozen_CIFAR10_Seed{seed}"
    row = df[df['project_name'] == project_name].iloc[-1]
    energy = row['energy_consumed']  # kWh
    results_with_energy.append((seed, emissions, elapsed, flops, energy))

print("\n--- Results Summary ---")
for seed, emissions, elapsed, flops, energy in results_with_energy:
    print(f"Seed {seed}: CO₂ = {emissions:.6f} kg, Time = {elapsed/60:.2f} min, FLOPs = {flops}, Energy = {energy:.6f} kWh")


--- Results Summary ---
Seed 0: CO₂ = 0.009835 kg, Time = 15.53 min, FLOPs = 0.044 GFLOPs, Energy = 0.028161 kWh
Seed 1: CO₂ = 0.009665 kg, Time = 15.18 min, FLOPs = 0.044 GFLOPs, Energy = 0.027674 kWh
Seed 2: CO₂ = 0.009702 kg, Time = 15.24 min, FLOPs = 0.044 GFLOPs, Energy = 0.027780 kWh
Seed 3: CO₂ = 0.009722 kg, Time = 15.27 min, FLOPs = 0.044 GFLOPs, Energy = 0.027839 kWh
Seed 4: CO₂ = 0.009772 kg, Time = 15.37 min, FLOPs = 0.044 GFLOPs, Energy = 0.027983 kWh


In [9]:
# Summary Statistics
co2 = np.array([r[1] for r in results_with_energy])
time_min = np.array([r[2]/60 for r in results_with_energy])
energy = np.array([r[4] for r in results_with_energy])

co2_mean, co2_std = co2.mean(), co2.std()
time_mean, time_std = time_min.mean(), time_min.std()
energy_mean, energy_std = energy.mean(), energy.std()

print("\n--- Summary Statistics ---")
print(f"CO₂ Emissions: Mean = {co2_mean:.6f} kg, Std Dev = {co2_std:.6f} kg")
print(f"Training Time: Mean = {time_mean:.2f} min, Std Dev = {time_std:.2f} min")
print(f"Energy Consumption: Mean = {energy_mean:.6f} kWh, Std Dev = {energy_std:.6f} kWh")


--- Summary Statistics ---
CO₂ Emissions: Mean = 0.009739 kg, Std Dev = 0.000059 kg
Training Time: Mean = 15.32 min, Std Dev = 0.12 min
Energy Consumption: Mean = 0.027887 kWh, Std Dev = 0.000169 kWh
